In [1]:
%cd ~/protein-transfer

/home/francesca/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
#!pip install receptivefield

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 KB 15.1 MB/s eta 0:00:00


In [45]:
from receptivefield.pytorch import PytorchReceptiveField

In [3]:
from sequence_models.pretrained import load_model_and_alphabet

model, collater = load_model_and_alphabet('carp_600k')

In [6]:
model.model.embedder.embedder

Embedding(30, 8, padding_idx=28)

In [24]:
model.model.embedder.up_embedder

PositionFeedForward(
  (conv): Conv1d(8, 128, kernel_size=(1,), stride=(1,))
)

In [9]:
model

CARP(
  (model): ByteNetLM(
    (embedder): ByteNet(
      (embedder): Embedding(30, 8, padding_idx=28)
      (up_embedder): PositionFeedForward(
        (conv): Conv1d(8, 128, kernel_size=(1,), stride=(1,))
      )
      (layers): ModuleList(
        (0): ByteNetBlock(
          (conv): MaskedConv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,))
          (sequence1): Sequential(
            (0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (1): GELU(approximate=none)
            (2): PositionFeedForward(
              (conv): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
            )
            (3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            (4): GELU(approximate=none)
          )
          (sequence2): Sequential(
            (0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            (1): GELU(approximate=none)
            (2): PositionFeedForward(
              (conv): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    

In [7]:
model.model.embedder.up_embedder

PositionFeedForward(
  (conv): Conv1d(8, 128, kernel_size=(1,), stride=(1,))
)

In [8]:
model.model.embedder.layers[0]

ByteNetBlock(
  (conv): MaskedConv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,))
  (sequence1): Sequential(
    (0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (1): GELU(approximate=none)
    (2): PositionFeedForward(
      (conv): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
    )
    (3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (4): GELU(approximate=none)
  )
  (sequence2): Sequential(
    (0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (1): GELU(approximate=none)
    (2): PositionFeedForward(
      (conv): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    )
  )
)

In [46]:
rf = PytorchReceptiveField(model.model.embedder)
rf_params = rf.compute(input_shape = (2,9))

TypeError: forward() missing 1 required positional argument: 'x'

In [5]:
# from scr.model.receptive_field import receptive_field, receptive_field_for_unit

In [8]:
seqs = [['TIM'], ['FRANCESCA']]
x = collater(seqs)[0]  # (n, max_len)
rep = model(x)  # (n, max_len, d_model)

In [37]:
x

tensor([[16,  7, 10, 27, 27, 27, 27, 27, 27],
        [ 4, 14,  0, 11,  1,  3, 15,  1,  0]])

In [15]:
x.shape, rep["representations"][16].shape

(torch.Size([2, 9]), torch.Size([2, 9, 128]))

In [33]:
import torch
import torch.nn as nn

In [38]:
embedder = nn.Embedding(num_embeddings=30, embedding_dim=8, padding_idx=28)

In [39]:
post_embed = embedder(x)
post_embed.shape

torch.Size([2, 9, 8])

In [40]:
up_embedder = nn.Conv1d(
        in_channels=8,
        out_channels=128,
        kernel_size=(1,),
        stride=(1,),
        padding=0,
        dilation=1,
        groups=1,
        bias=True,
        padding_mode="zeros",
        device=None,
        dtype=None,
    )

In [41]:
post_upemb = up_embedder(post_embed.transpose(1, 2)).transpose(1, 2)
post_upemb.shape

torch.Size([2, 9, 128])

In [43]:
model.model.embedder.layers[0]

ByteNetBlock(
  (conv): MaskedConv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,))
  (sequence1): Sequential(
    (0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (1): GELU(approximate=none)
    (2): PositionFeedForward(
      (conv): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
    )
    (3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (4): GELU(approximate=none)
  )
  (sequence2): Sequential(
    (0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (1): GELU(approximate=none)
    (2): PositionFeedForward(
      (conv): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    )
  )
)

In [18]:

receptive_field_dict = receptive_field(model.model.embedder.layers[0].to("cuda"), input_size=(1, 8, 1280))
# receptive_field_for_unit(receptive_field_dict, "1", (1,1))

m_key: 1, p_key: 0
receptive_field: OrderedDict([('0', OrderedDict([('j', 1.0), ('r', 1.0), ('start', 0.5), ('conv_stage', True), ('output_shape', [-1, 1, 8, 1280])])), ('1', OrderedDict())])
receptive_field[p_key]: OrderedDict([('j', 1.0), ('r', 1.0), ('start', 0.5), ('conv_stage', True), ('output_shape', [-1, 1, 8, 1280])])
m_key: 2, p_key: 1
receptive_field: OrderedDict([('0', OrderedDict([('j', 1.0), ('r', 1.0), ('start', 0.5), ('conv_stage', True), ('output_shape', [-1, 1, 8, 1280])])), ('1', OrderedDict([('input_shape', [-1, 1, 8, 1280]), ('output_shape', [-1, 1, 8, 1280])])), ('2', OrderedDict())])
receptive_field[p_key]: OrderedDict([('input_shape', [-1, 1, 8, 1280]), ('output_shape', [-1, 1, 8, 1280])])


KeyError: 'j'